In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import pymc as pm
import arviz as az


WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [8]:
from numpy.random import SeedSequence, default_rng

# 1. Make a master SeedSequence
ss = SeedSequence(12345)

# 2. Spawn N child SeedSequences
children = ss.spawn(4)  

# 3. Build independent Generator objects from each child
rngs = [default_rng(child) for child in children]

# Now rngs[0], rngs[1], … are independent RNGs you can use.


In [9]:
# read in the data
## this is the data from the demo
ano_counts_month = pd.read_csv('Demo_counts_by_month_csv.csv')

In [11]:
ano_counts_month

,year,month,AN,NN,BN,ENSO
0,2000,1,14,10,21,L
1,2000,2,7,10,10,N
2,2000,3,17,13,8,N
3,2000,4,10,9,11,N
4,2000,5,15,8,14,N
5,2000,6,7,10,3,N
6,2000,7,9,4,16,L
7,2000,8,9,12,6,L
8,2000,9,6,6,12,N
9,2000,10,10,6,17,L


In [10]:

# 1. ─── Load your data ─────────────────────────────────────────────────────────
# Expect a CSV (or similar) with one row per month, columns:
#   'year','month','AN','NN','BN','ENSO'
# where 'AN','NN','BN' are the counts of above‐normal, near‐normal, below‐normal days,
# and 'ENSO' is one of {'E','N','L'}.
df = ano_counts_month

# map ENSO state to integer index 0=El Niño, 1=Neutral, 2=La Niña
enso_map = {'E': 0, 'N': 1, 'L': 2}
s_idx = df['ENSO'].map(enso_map).values  # shape (N,)

# observed counts array of shape (N, 3)
y = df[['AN','NN','BN']].values

# number of seasons/months
N = y.shape[0]

# 2. ─── Specify prior hyperparameters ────────────────────────────────────────
# αs,c and βs,c for s in {E,N,L}, c in {AN,NN,BN}.
# Fill these with your chosen prior strengths.
alpha_prior = np.array([
    [2.0, 2.0, 2.0],   # for El Niño: α_{E,AN}, α_{E,NN}, α_{E,BN}
    [2.0, 2.0, 2.0],   # for Neutral
    [2.0, 2.0, 2.0],   # for La Niña
])
beta_prior  = np.array([
    [1.0, 1.0, 1.0],   # for El Niño: β_{E,AN}, β_{E,NN}, β_{E,BN}
    [1.0, 1.0, 1.0],   # for Neutral
    [1.0, 1.0, 1.0],   # for La Niña
])

# 3. ─── Build and sample from the model ─────────────────────────────────────
with pm.Model() as enso_count_model:

    # a) Poisson rates λ_{s,c} with Gamma(α,β) priors
    #    shape=(3,3): first index = ENSO state, second = category c
    lambda_sc = pm.Gamma(
        "lambda_sc",
        alpha=alpha_prior,
        beta=beta_prior,
        shape=(3,3)
    )

    # b) For each month i, pick the row lambda_sc[s_idx[i], :] → shape (3,)
    mu = lambda_sc[s_idx]

    # c) Likelihood: independent Poisson draws for the three categories
    y_obs = pm.Poisson(
        "y_obs",
        mu=mu,
        observed=y
    )

    # d) Run the MCMC
    trace = pm.sample(
        draws=2000,        # number of posterior samples
        tune=1000,         # burn‐in
        target_accept=0.9, # increase if you see divergences
        return_inferencedata=True
    )

# 4. ─── Diagnostics and summary ──────────────────────────────────────────────
# Trace plots for each λ_{s,c}
az.plot_trace(trace, var_names=["lambda_sc"])
az.summary(trace, var_names=["lambda_sc"])


AttributeError: 'numpy.random._generator.Generator' object has no attribute 'spawn'